In [ ]:
%matplotlib inline
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
small_tick_size = 8
small_label_size = 8

# Tutorial 2: Equiprobability, partition function

## Equiprobability principle

As mentionned in the lecture, the system of hard disks in a box with elastic collisions is typical of a microcanonical thermodynamic ensemble. As such, the statistical weight of each configuration depends on its energy, which is the same for all possible configurations. For any two configurations $a$ and $b$, we have

$$
\pi(a) = \pi[E(a)] = \pi[E(b)] = \pi(b)
$$

It is therefore necessary that the Monte Carlo algorithm sample these configurations with equal probabilities. In the lecture, an example of direct sampling Monte Carlo algorithm was introduced. It was then stated that the "tabula rasa" mechanism should be used. Instead, one could consider a different way of handling accidental overlaps: as soon as an overlap occurs, it could be possible to remove the disk causing the overlap, and look for a suitable position for it, until we reach success. This approach is called "random sequential deposition". The fact is that the latter approach does not produce configurations with equal probability, as we will now see.

### Discrete hard rods model: random sequential deposition

The discrete hard rods model is a 1D, discrete version of the hard disks model. Considering rods of length 2 + $\epsilon$, it is obvious that for a configuration to be valid, the centers of each pair of rads need to be separated by 3 units or more. In other terms, their centers must be separated by at least two free sites.

Let us consider this model using five hooks $[0, \ldots, 4]$, two rods, and compare the tabula rasa with the random sequential deposition. With the random sequential deposition, the first rod has a probability $1/5$ to be dropped on one of the five sites. If it is dropped onto site #2, then it is not possible to find a suitable opsition for the second rod. Therefore, the position of the first site must be confined to the set ${0, 1, 3, 4}$, with a probability $1/4$. It is clear that if the first rod is dropped on site 0 (resp. 4), then there are two possible suitable positions for the second rod: ${3,4}$ (resp ${0, 1}$. This gives a probability $1/8$ for each of these possible configurations. Conversely, if the first rod is dropped on site 1 (resp. 3), then there is only one acceptable opsition for the second rod: site 4 (resp 0). Each of these configurations therefore occurs with probability $1/4$.

We have therefore shown that this scheme does not respect the equiprobability principle. The implementation in the code below shows that indeed, the configurations have different probabilities.

In [ ]:
configurations = {(0, 3): 'a', (0, 4): 'b', (1, 4): 'c', 
                  (3, 0): 'd', (4, 0): 'e', (4, 1): 'f'}
counts = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0}

n_runs = 10000

for i in range(n_runs):
    red_rod = random.randint(0, 3)
    if red_rod >= 2:
        red_rod += 1
    while True:
        blue_rod = random.randint(0, 4)
        if abs(red_rod - blue_rod) > 2:
            break
    conf = configurations[(red_rod, blue_rod)]  
    counts[conf] += 1
for conf in counts: 
    print conf, counts[conf] / float(n_runs)    


### Discrete hard rods model: tabula rasa

The code shown below implements the tabula rasa approach for the discrete hard rods model.

In [ ]:
configurations = {(0, 3): 'a', (0, 4): 'b', (1, 4): 'c',
                  (3, 0): 'd', (4, 0): 'e', (4, 1): 'f'}
counts = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0}
n_runs = 10000
for run in range(n_runs):
    while True:
        red_rod = random.randint(0, 4)
        blue_rod = random.randint(0, 4)
        if abs(red_rod - blue_rod) > 2: break
    conf = configurations[(red_rod, blue_rod)]
    counts[conf] += 1
for conf in counts:
    print conf, counts[conf] / float(n_runs)


### Questions:

1. How many different configurations, legal and illegal, can the code above generate?

2. Among all these configurations, how many are legal?

3. With what statistical weight does the tabula rasa algorithm produce each legal configuration?

The argument used for the hard rods applies equally to the hard disks. The code snippet below implements the tabula rasa rule for hard disks: it generates the position of all disks, before it checks if the configuration is legal. The vector L is a eight-dimensional vector in an eight-dimensional hypercube (between $\sigma$ and $1-\sigma$ in each dimension), whose inner points are sampled with equal probability. Then, the volume of the hypercube which corresponds to illegal configurations is discarded, as was the case in the 1D hard rods model.

In [ ]:
N = 4
sigma = 0.2
pairs = [(i, j) for i in range(N - 1) for j in range(i + 1, N)]
counter = 0
while True:
    counter += 1
    L = [(random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma)) for k in range(N)]
    if  min((L[i][0] - L[j][0]) ** 2 + (L[i][1] - L[j][1]) ** 2 for i, j in pairs) > 4.0 * sigma ** 2: 
        break

print "nb of trials: ", counter
print "configuration: "
for elem in L:
    print elem

## Partition function: numerical approach

The partition function is an essential quantity in statistical physics. In order to compute the partition function of the system of hard disks, we will get rid of the wall collisions, and implement periodic boundary conditions, in which disks can be split and wrap around the limits of the box. This implies that two disks which may seem far apart may actually overlap, e.g. a disk in the upper left corner and a disk in the lower left corner. The code below implements the direct sampling method, modified for the periodic boundary conditions. It also produces figures representing the legal configurations which are found.

### Questions

1. Make sure you understand the "dist" function implementation in this new framework.
2. Note that in the following code, the quantity eta has been introduced, defined as the actual fraction of the box surface which is occupied by the disks. Note also that the periodic boundary conditions allow us to pick random coordinates between 0 and 1, instead of $\sigma$ and $a-\sigma$.
3. Comment on the number of wipe-outs which are executed, as a function of eta, as eta approaches its theoretical maximum, and as the number of disks increases (check $N=16$, and $\eta \to 0.3$). Please also assess the visual aspect of the disk occupancy (see figures generated by the following code). What do you think of this algorithm?

In [ ]:
def dist(x,y):
    d_x = abs(x[0] - y[0]) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(x[1] - y[1]) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  math.sqrt(d_x**2 + d_y**2)
    
def direct_disks(N, sigma):
    n_iter = 0
    condition = False
    while condition == False:
        n_iter += 1
        L = [(random.random(), random.random())]
        for k in range(1, N):
            a = (random.random(), random.random())
            min_dist = min(dist(a, b) for b in L) 
            if min_dist < 2.0 * sigma: 
                condition = False
                break
            else:
                L.append(a)
                condition = True
    return n_iter, L

img = 0
output_dir = 'F_7110_direct_disks_multirun'
if not os.path.exists(output_dir): os.makedirs(output_dir)

def snapshot(pos, colors, border_color = 'k'):
    global img
    pylab.figure()
    pylab.axis([0, 1, 0, 1])
    [i.set_linewidth(2) for i in pylab.gca().spines.itervalues()]
    [i.set_color(border_color) for i in pylab.gca().spines.itervalues()]
    pylab.setp(pylab.gca(), xticks = [0, 1], yticks = [0, 1], aspect = 'equal')
    for (x, y), c in zip(pos, colors):
        circle = pylab.Circle((x, y), radius = sigma, fc = c)
        pylab.gca().add_patch(circle)
    pylab.savefig(output_dir+'/snapshot_%03i.png'%img)
    pylab.close()
    img += 1

def periodicize(config):
    images = [-1.0, 0.0, 1.0]
    return [(x + dx, y + dy) for (x,y) in config for dx in images for dy in images]

N = 4
eta = 0.1
sigma = math.sqrt(eta / N / math.pi)
n_runs = 8
if n_runs > 10:
    print ('Note that this program might take a while! - n_runs is limited to 10')
    n_runs = min(n_runs, 10)

colors = ['r' for i in range(8 * N)]
for run in range(n_runs):
    iterations, config =  direct_disks(N, sigma)
    print 'run', run
    print iterations - 1, 'tabula rasa wipe-outs before producing the following configuration'
    print config
    print
    config_per = periodicize(config)
    snapshot(config_per, colors, border_color = 'k')

Our algorithm is affected by the so-called "curse of dimensionality": assuming that the considered box has a volume $V$, the volume of the space which the algorithm has to sample with homogeneous probability is $V^N$ where $N$ is the number of disks. For $\eta=0$, all configurations are legal, i.e. $Z(\eta=0) = \int_{V} dx_0 \ldots \int_{V} dx_{N-1} = V^N$, and $p_{accept} = 1$. Conversely, for finite values of $\eta$, the code above allows us to measure that the acceptance ratio drops to $p \simeq 5.10^{-6}$ for eta=0.3. The volume of acceptable configurations is tiny, compared to the volume of points to be sampled, and a huge fraction of our random configuration draws fails.

This code allows us to evaluate numerically the function $p_{accept}(\eta)$, as will be illustrated in the next section. When the calculation is carried out, it is observed that $p_{accept}(\eta)$ decays faster than exponentially with $\eta$. In the following section, we will confirm this numerical conclusion by carrying out an analytical expansion of the acceptance ratio in powers of $\Upsilon(x_i, x_j)$, defined as the Indicator function: $\Upsilon(x_i, x_j) = \delta[dist(x_i, x_j)<2\sigma]$. This expansion is exact in the limit of vanishing density.

## Partition function: virial expansion

We will derive the following expression in class, which is the first order virial expansion of the partition function:

$$ Z(\eta) \simeq Z(0) \exp [-2(N-1)\eta]$$

This shows that the acceptance ratio diminishes with both $\eta$ **and** $N$. The expansion can be transformed, and superior orders can be taken into account, leading to an expansion in powers of $1/V$:

$$ \dfrac{V}{N} \dfrac{\partial \log Z(\eta)}{\partial V} = 1 + 2(N-1)\pi \sigma^2\frac{1}{V}  + \mathcal{O}[\dfrac{1}{V^2}]$$


The first term on the right is a constant, and corresponds to the case of the ideal gas: a system of disks of radius 0. The second term, in $\dfrac{1}{V}$, corresponds to the expansion limited to first order in $\Upsilon$. The following terms are generated by the higher orders of $\Upsilon$.

We can compare this analytical approximation with the numerical result, with the code given below.

**Question**: Do you understand how this code can estimate the profile of $p_\text{acceptance}$ ?

In [ ]:
def dist(x, y):
    d_x = abs(x[0] - y[0]) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(x[1] - y[1]) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  math.sqrt(d_x**2 + d_y**2)
    
N = 16
n_confs = 10 ** 5
pairs = [(i, j) for i in range(N - 1) for j in range(i + 1, N)]
eta_max_list = []
for conf in xrange(n_confs):
    L = [(random.random(), random.random()) for k in range(N)]
    sigma_max = min(dist(L[i], L[j]) for i, j in pairs) / 2.0
    eta_max = N * math.pi * sigma_max ** 2
    eta_max_list.append(eta_max)

# Begin of graphics output
pylab.figure()
n, bins, patches = pylab.hist(eta_max_list, 100, histtype='step', cumulative=-1, 
                   log=True, normed=True, label="numerical evaluation of p_accept")
explaw = [math.exp( - 2.0 * (N - 1) * eta) for eta in bins]
pylab.plot(bins, explaw, 'r--', linewidth=1.5, label="1st order virial expansion")
pylab.xlabel('density eta')
pylab.ylabel('p_accept(eta)')
pylab.legend()
pylab.show()

As expected, the fit is very good in the limit of low densities, and the exact acceptance ratio decays faster than exponentially. This expansion dates back to the work of L. Boltzmann, who carried it out up to fourth order. It was once believed that this approximation was a way to handle the thermodynamics of hard disks in a complete way. Unfortunately, it was later confirmed that the convergence radius of this expansion is finite, and that this expansion does not predict any phase transition. Such transitions will be studied in the next weeks. In particular, we will see how our algorithms can take us beyong the limitations of perturbative approaches, and allow us to observe phase transitions to the solid phase.